<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [26]:

headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                         "(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

response = requests.get(static_url, headers=headers)

print(response.status_code)  # should now be 200
soup = BeautifulSoup(response.text, "html.parser")


200


Create a `BeautifulSoup` object from the HTML `response`


In [19]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [27]:
# Use soup.title attribute
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [32]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
# Find all table elements in the HTML
html_tables = soup.find_all("table")



Starting from the third table is our target table contains the actual launch records.


In [33]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [34]:
# --- assume you already have `soup` ---
# If you previously got 403s, make sure you set a proper User-Agent when creating `soup`.

# 1) Collect tables safely
html_tables = soup.find_all("table")
print(f"Found {len(html_tables)} tables")

if not html_tables:
    raise RuntimeError("No <table> elements found. Check that the request returned HTML (status 200) and not a 403/redirect.")

# 2) Prefer Wikipedia 'wikitable' tables if present
wikitable_candidates = soup.select("table.wikitable")
tables_to_search = wikitable_candidates or html_tables

# 3) Pick the first table whose headers look relevant
def table_headers(tbl):
    return [th.get_text(strip=True) for th in tbl.find_all("th")]

first_launch_table = None
for tbl in tables_to_search:
    hdrs = table_headers(tbl)
    if any(("Launch" in h or "Booster" in h or "Payload" in h) for h in hdrs):
        first_launch_table = tbl
        break

# fallback to the first table if nothing matched
first_launch_table = first_launch_table or tables_to_search[0]

# 4) Find the first row that actually has <th> cells (covers <thead> and plain header rows)
header_row = next((tr for tr in first_launch_table.find_all("tr") if tr.find("th")), None)
if header_row is None:
    raise RuntimeError("No header row (<th>) found in the selected table.")

ths = header_row.find_all("th")

# 5) Extract clean header names using provided helper if available; fallback to plain text
column_names = []
for th in ths:
    try:
        name = extract_column_from_header(th)  # provided by your lab
    except NameError:
        name = th.get_text(strip=True)  # fallback if helper isn't defined

    if name:
        column_names.append(name)

print("Column names:", column_names)

# 6) Optional: sanity checks / debug
if not column_names:
    # Print raw header texts to help debug if extraction wiped everything
    raw = [th.get_text(strip=True) for th in ths]
    print("No column names extracted; raw header texts:", raw)


Found 25 tables
Column names: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Check the extracted column names


In [35]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [36]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [38]:
extracted_row = 0

# Extract each table
for table_number, table in enumerate(soup.find_all("table", "wikitable plainrowheaders collapsible")):
    # get table rows
    for rows in table.find_all("tr"):
        # check if first table heading is a number corresponding to launch number
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()
        else:
            flag = False

        # get table cells
        row = rows.find_all("td")

        # if it is a number, save cells in a dictionary
        if flag:
            extracted_row += 1

            # Flight Number
            # launch_dict["Flight No."].append(flight_number)

            # Date / Time
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(",")
            time = datatimelist[1]

            # Booster version
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].get_text(strip=True)

            # Launch Site
            launch_site = row[2].get_text(strip=True)

            # Payload
            payload = row[3].get_text(strip=True)

            # Payload Mass
            payload_mass = get_mass(row[4])

            # Orbit
            orbit = row[5].get_text(strip=True)

            # Customer
            customer = row[6].get_text(strip=True)

            # Launch outcome
            launch_outcome = row[7].get_text(strip=True)

            # Booster landing
            booster_landing = landing_status(row[8])

            # For debugging
            print(f"Row {extracted_row}:",
                  flight_number, date, time, bv, launch_site,
                  payload, payload_mass, orbit, customer,
                  launch_outcome, booster_landing)

        

Row 1: 1 4 June 2010 18:45 F9 v1.07B0003.18 CCAFS,SLC-40 Dragon Spacecraft Qualification Unit 0 LEO SpaceX Success Failure
Row 2: 2 8 December 2010 15:43 F9 v1.07B0004.18 CCAFS,SLC-40 Dragondemo flight C1(Dragon C101) 0 LEO(ISS) NASA(COTS)NRO Success[9] Failure
Row 3: 3 22 May 2012 07:44 F9 v1.07B0005.18 CCAFS,SLC-40 Dragondemo flight C2+[18](Dragon C102) 525 kg LEO(ISS) NASA(COTS) Success[20] No attempt

Row 4: 4 8 October 2012 00:35 F9 v1.07B0006.18 CCAFS,SLC-40 SpaceX CRS-1[22](Dragon C103) 4,700 kg LEO(ISS) NASA(CRS) Success No attempt
Row 5: 5 1 March 2013 15:10 F9 v1.07B0007.18 CCAFS,SLC-40 SpaceX CRS-2[22](Dragon C104) 4,877 kg LEO(ISS) NASA(CRS) Success No attempt

Row 6: 6 29 September 2013 16:00 F9 v1.17B10038 VAFB,SLC-4E CASSIOPE[22][31] 500 kg Polar orbitLEO MDA Success[30] Uncontrolled
Row 7: 7 3 December 2013 22:41 F9 v1.1B1004 CCAFS,SLC-40 SES-8[22][35][36] 3,170 kg GTO SES Success[37] No attempt
Row 8: 8 6 January 2014 22:06 F9 v1.1 CCAFS,SLC-40 Thaicom 6[22] 3,325 kg G

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [40]:
df = pd.DataFrame({ 
    key: pd.Series(value, dtype="object") for key, value in launch_dict.items() 
})


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
